In [ ]:
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import skimage.io
import skimage
import cv2
import tifffile as tiff 
import numpy as np
import tifffile as tiff
import xmltodict as xd, numpy as np, pickle
from collections import defaultdict
import pandas as pd
import matplotlib, matplotlib.pyplot as plt
matplotlib.rcParams['figure.dpi']=300
import seaborn as sns
from scipy.interpolate import splprep, splev
import matplotlib.pyplot as plt

def fit_spline(pts):
    try:
        okay = np.where(np.abs(np.diff(pts,axis=0)).sum(1) > 0)[0]
        pts = np.vstack([pts[okay], pts[-1]])#, pts[0]

        tck, u = splprep(pts.T, u=None, s=0.0, per=1) 
        u_new = np.linspace(u.min(), u.max(), 1000)
        x_new, y_new = splev(u_new, tck, der=0)
        pts=np.vstack((x_new, y_new)).T
    except:
        pass
    return pts


def gen_contour_df(xml,compression=1,include_labels=[]):
    with open(xml,"rb") as f:
            d=xd.parse(f)

    cells=[]
    cell_dot=[]
    lbls=[]
    for i,annotation in enumerate(d['ASAP_Annotations']["Annotations"]["Annotation"]):
        try:
            lbl=annotation["@PartOfGroup"]
            contour=np.array([(float(coord["@X"]),float(coord["@Y"])) for coord in annotation["Coordinates"]["Coordinate"]])
            cells.append(contour)
            lbls.append(lbl)
        except:
            print(i)

    contour_df=pd.DataFrame(cells,columns=['contours'])
    contour_df['contours']=contour_df['contours'].map(lambda x:x/compression)
    contour_df['xmin']=contour_df['contours'].map(lambda x: x[:,0].min())
    contour_df['xmax']=contour_df['contours'].map(lambda x: x[:,0].max())
    contour_df['ymin']=contour_df['contours'].map(lambda x: x[:,1].min())
    contour_df['ymax']=contour_df['contours'].map(lambda x: x[:,1].max())
    contour_df['xmean']=contour_df['contours'].map(lambda x: x[:,0].mean())
    contour_df['ymean']=contour_df['contours'].map(lambda x: x[:,1].mean())
    contour_df['lbl']=[lbl.lower() for lbl in lbls] # .split()[0]
    contour_df.loc[:,'contours']=contour_df['contours'].map(fit_spline)
    if include_labels:
        contour_df=contour_df[contour_df['lbl'].isin(include_labels)]

    return contour_df

In [ ]:
! ls new_skin_layers/Skin_Layer_XML/*.xml

In [ ]:
import glob,os
annots=dict()
annots['tumor']={os.path.basename(f).replace("_ASAP","").replace(".xml",""):gen_contour_df(f) for f in glob.glob("annotations/BCC/BCC XML/*.xml")}
annots['macro']={os.path.basename(f).replace("_ASAP","").replace(".xml",""):gen_contour_df(f) for f in glob.glob("new_skin_layers/Skin_Layer_XML/*.xml")}



In [ ]:
annots['tumor']['3_A2d']=annots['tumor']['242125_3 A2d']
annots['tumor']['264_A1a']=annots['tumor']['264 edited_A1']

del annots['tumor']['242125_3 A2d'], annots['tumor']['264 edited_A1']

In [ ]:
for k in annots:
    for k2 in annots[k]:
        annots[k][k2]['ID']=k2

In [ ]:
df_annot=dict(tumor=[],macro=[])
for k in df_annot:
    df_annot[k]=pd.concat(list(annots[k].values()))

In [ ]:
df_annot['macro']=df_annot['macro'][df_annot['macro']['lbl'].isin(['dermis', 'epidermis',
                                        'subcutaneous tissue',
                                       ])]#.unique()

In [ ]:
df={}
for k in df_annot:
    df[k]={os.path.basename(f).replace('.pkl','') : pd.read_pickle(f) for f in glob.glob(f"patch_info/{k}/*.pkl")}
    for k2 in df[k]:
        df[k][k2]['ID']==k2
    df[k]=pd.concat(list(df[k].values()))

In [ ]:
import numpy as np
from pathflowai.utils import generate_tissue_mask
from skimage import draw
import numpy as np
import cv2
compression=8
patch_size=256

def get_dff(k, k2='macro'):
    try:
        arr=cv2.resize(np.load(f"./inputs/{k}.npy"),None,fx=1/compression,fy=1/compression)
        if k2=='macro':
            mask=generate_tissue_mask(arr,compression=compression/2,threshold=230, keep_holes=False, max_hole_size=200, kernel=5)
            mask2=generate_tissue_mask(arr,compression=compression/2,threshold=230, keep_holes=False, max_hole_size=200000, kernel=5)
            mask3=mask2^mask
        dff=df[k2][df[k2]['ID']==k]
        if k2=='macro': dff['hole']=[mask3[x:x+int(patch_size/compression),y:y+int(patch_size/compression)].astype(float).mean() for x,y in (dff[['x','y']].values/compression).astype(int).tolist()]
        for name, annots_ in annots[k2][k].groupby('lbl'):
            annots_=cv2.fillPoly(np.zeros(arr.shape[:2]), pts=annots_['contours'].map(lambda x:(x/compression).astype(int)).tolist(), color=1)
            dff[name]=[annots_[x:x+int(patch_size/compression),y:y+int(patch_size/compression)].astype(float).mean() for x,y in (dff[['x','y']].values/compression).astype(int).tolist()]
    #     tissue_types=['epidermis','dermis','subcutaneous tissue','hole']
    #     for k in tissue_types:
    #         if k not in dff.columns: dff[k]=0.
    #     dff.loc[dff[tissue_types].sum(1)==0,'annotation']='unassigned'
    #     for k in tissue_types:
    #         dff.loc[dff[k]>0.0,'annotation']=k
        if k2=='macro': del mask, mask2, mask3
        del arr
    except:
        dff=[]
    return dff

In [ ]:
import pickle
df_update={}
for k in ['tumor','macro']:
    IDs=set(annots[k].keys()).intersection(set(df[k]['ID'].unique()))
    df_update[k]=pd.concat([x for x in [get_dff(ID,k) for ID in IDs] if x is not []])
pickle.dump(df_update,open('model_patch_annotations/prelim_update.pkl','wb'))

In [ ]:
import pickle
df_update=pickle.load(open('model_patch_annotations/prelim_update.pkl','rb'))

In [ ]:
df_tumor=df_update['tumor'][['ID', 'x', 'y', 'patch_size', 'annotation', 'bcc','inflammation','inflamation']].fillna(0)
df_tumor['inflammation']+=df_tumor['inflamation']
df_tumor=df_tumor.drop(columns=['inflamation'])

In [ ]:
df_tumor['annotation']='benign'
for k in ['inflammation','bcc']:
    df_tumor.loc[df_tumor[k]>0.,'annotation']=k

In [ ]:
df_macro=df_update['macro'].copy()
tissue_types=['dermis','hole','subcutaneous tissue','epidermis']
for k in tissue_types:
    if k not in df_macro.columns: df_macro[k]=0.
df_macro.loc[df_macro[tissue_types].sum(1)==0,'annotation']='unassigned'
for k in tissue_types:
    df_macro.loc[df_macro[k]>(0.0 if k!= "hole" else 0.5),'annotation']=k
df_macro.loc[df_macro['annotation']=="0","annotation"]="unassigned"
df_macro=df_macro[['ID', 'x', 'y', 'patch_size', 'annotation']+tissue_types]
# change this scheme in the future!! whispy derm (eg. hole in derm unannoted -> unassigned for CNN; then to derm annotation in gnn)

In [ ]:
import cv2, numpy as np, seaborn as sns, matplotlib, matplotlib.pyplot as plt
matplotlib.rcParams['figure.dpi']=300
k="78_A2d"
arr=cv2.resize(np.load(f"inputs/{k}.npy"),None,fx=1/8,fy=1/8)
df_anno=df_macro.copy()
plt.subplot(121)
plt.imshow(np.transpose(arr,(1,0,2)))
plt.subplot(122)
plt.imshow(np.transpose(arr,(1,0,2)))
import seaborn as sns
# for name,dff in val_data['patch_info'].groupby('ID'):#
dff=df_anno[df_anno['ID']==k]
# plt.figure()
dff[['x','y']]=dff[['y','x']].iloc[:,::-1]/8
sns.scatterplot(x='x',y='y',hue='annotation',data=dff,s=2.,alpha=1)#"hole"
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title(k)
# tissue mask blurring removing some obvious holes

In [3]:
# ! mkdir old_slides && mv new_test_slides/'ASAP Tiff'/'264 edited_A1_ASAP.tif' old_slides

In [4]:
# ! mv "new_test_slides/ASAP Tiff" "new_test_slides/ASAP_Tiff"

In [ ]:
import tifffile
im=tifffile.imread("new_test_slides/ASAP_Tiff/100_A1b_ASAP.tif")
# ! ls new_test_slides/* -lah 

In [ ]:
! realpath ../../cgat/inputs/


In [ ]:
im.shape

In [ ]:
import numpy as np
np.load("inputs/100_A1b.npy").shape

In [ ]:
! ls inputs

In [ ]:
df_macro.to_pickle("model_patch_annotations/macro_annot.pkl")

In [ ]:
df_tumor.to_pickle("model_patch_annotations/tumor_annot.pkl")

In [ ]:
df_macro['annotation'].value_counts()

In [ ]:
df_macro['ID'].nunique()

In [ ]:
df_tumor['ID'].nunique()

In [ ]:
import seaborn as sns
sns.scatterplot(x='x',y='y',hue='annotation',data=df_macro[df_macro['ID']==df_macro['ID'].unique()[7]],s=1)

In [ ]:
sns.scatterplot(x='x',y='y',hue='annotation',data=df_tumor[df_tumor['ID']==df_tumor['ID'].unique()[15]],s=1)

In [ ]:
# remove cases with significant amount unassigned; indicates faulty xml ^^